## 1. OpenCV DNN Object Counting
- Intro to Object Counting
- Simple Object Counting using OpenCV DNN
- Object Counting in Region of Interest (ROI)

### 1.1 Intro Object Counting

- Object counting in specific zone or region of interest <br>
<img src="resource/oc4.gif" width="600px"> <br>
source : https://blog.roboflow.com/how-to-count-objects-in-a-zone/<br><br><br>

### 1.2 Simple Object Counting using OpenCV DNN

In [ ]:
import cv2 
import numpy as np
import utils 

cv2.__version__

- since we are suing original `yolov8m.pt` trained using `COCO dataset`, we will load `classNames` from `coco.py`
- If you are using your own model with custom dataset, you just simply define classsNames in format,
    ```
    classNames = ["scissors", "paper", "cat"]
    ```

In [ ]:
import coco

classNames = coco.load_coco_class_names_yolo()

print(classNames)

#### 1.2.1 Download Yolo V8 model

- For this case we will using Yolo V8 model for object detection
- Download and Convert Yolo V8 model using colab notebook [Convert Pytorch Model (.pt) to ONNX.ipynb](https://colab.research.google.com/drive/1IDHaSJyIauPgI_TE9UXHLm2m2nUUaKXb)
- Put the downloaded `yolov8s.onnx` into `model/` directory

#### 1.2.2 Utils.py modification to support simple object counting

- to draw count of object in prediction result we add extra function in `utils.py`
    - `utility.draw_object_count(img, x0, y0, font_size=0.4, color=(0,127,255), text_color=(255,255,255), padding = 10)`
    - where : 
        - `img` : input image
        - `x0` and `y0` : location of counting label to draw (top-left corner box in pixel)
        - `font_size=0.4, color=(0,127,255), text_color=(255,255,255)` : default font and color style
        - `padding = 10` : is padding between counting label (in pixel)

<br><br><img src="resource/preview.png" width="700px">

- we are also have `utility.print_object_count()` function to print detection result into text in format
```
'person' : 2
```

In [5]:
utility = utils.Utils()

# load trained Yolo V8s ONNX
model = "model/yolov8s.onnx"
net = cv2.dnn.readNetFromONNX(model)


# load sample image & convert to blob
img = cv2.imread("image1.jpg")


resize_h, resize_w = 320, 320 
blob = cv2.dnn.blobFromImage(img, 1/255.0, (resize_w, resize_h), (0, 0, 0), swapRB=True, crop=False)


# do forward pass (inferencing)
net.setInput(blob)
output = net.forward()

# do a postprocessing
img = utility.postprocess_onnx(output, img, classNames, confThreshold = 0.5, nmsThreshold = 0.2, font_size=0.3, 
                        color=(255,127,0), text_color=(255,255,255), input_size=[resize_h, resize_w])

# do print count of object
utility.print_object_count()

# do draw count of object in image
h, w, __ = img.shape
x0, y0 = int(0.75*w), int(0.9*h) # define counter box in 75% width , 10% height
img = utility.draw_object_count(img, x0, y0, font_size=0.4, color=(0,127,255), text_color=(255,255,255), padding = 10)


# show result 
cv2.imshow("detection result 1", img)
cv2.waitKey()
cv2.destroyAllWindows()

- Test using camera stream

In [ ]:
utility = utils.Utils()

# load trained Yolo V8 ONNX
model = "model/yolov8s.onnx"
net = cv2.dnn.readNetFromONNX(model)

# load video
cap = cv2.VideoCapture(0)
           
# iterate for each frame in video
while cap.isOpened():
    
    # get image on each frame
    ret, frame = cap.read()
    if not ret:
        break

    # convert to blob
    resize_h, resize_w = 320, 320 # use smaller image size to reduce computation 
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (resize_w, resize_h), (0, 0, 0), swapRB=True, crop=False)

    # do a net forward (inferencing)
    net.setInput(blob)
    output = net.forward()

    # do a postprocessing
    frame = utility.postprocess_onnx(output, frame, classNames, confThreshold = 0.6, nmsThreshold = 0.4, font_size=0.5, 
                        color=(255,127,0), text_color=(255,255,255), input_size=[resize_h, resize_w])

    # do print count of object
    utility.print_object_count()

    # do draw count of object in image
    h, w, __ = frame.shape
    x0, y0 = int(0.75*w), int(0.1*h) # define counter box in 75% width , 10% height
    frame = utility.draw_object_count(frame, x0, y0, font_size=0.4, color=(0,127,255), text_color=(255,255,255), padding = 10)

    # show result
    cv2.imshow('Frame',frame)

    # wait 25ms per frame and close using 'q' 
    if cv2.waitKey(1) == ord('q'):
          break


cap.release()
cv2.destroyAllWindows()

### 1.3 Object Counting in Region of Interest (ROI)

- Download sample video `mall.mp4` from https://drive.google.com/file/d/1VlwGdLnP52zs9vvRJjbOX-rWoZuSnZfv/view?usp=sharing
- put in `Pertemuan 6/` directory (*make sure the video in same folder with this notebook*)<br><br><br>
- We will count the object passing through the ROI like below<br>
<img src="resource/oc4.gif" width="500px"> <br><br>
- To find polygone of ROI we can use Roboflow *PolygonZone* (https://polygonzone.roboflow.com/)
<img src="resource/rb-pz.png" width="600px">

- Use below code to exctract specific image from video `mall.mp4`
    - press 'q' for close the video
    - press 's' for save the video frame into image `base_roi.jpg`, then close

In [ ]:
# load video mall.mp4
cap = cv2.VideoCapture("mall.mp4")
           
# iterate for each frame in video
while cap.isOpened():
    # get image on each frame
    ret, frame = cap.read()
    if not ret:
        break

    # show result
    cv2.imshow('Frame',frame)

    # wait 1ms per frame and close using 'q' and 's' for save
    key = cv2.waitKey(50)
    if key == ord('q'):
        break
    if key == ord('s'):
        cv2.imwrite("base_roi.jpg", frame)
        print("`base_roi.jpg` saved successfully!")
        print("Upload the saved image into https://polygonzone.roboflow.com/ to find poligon ROI")
        break


cap.release()
cv2.destroyAllWindows()

- Draw polygon of ROI, then
- copy the generated numpy array `points` on the left side
<img src="resource/rb-polygonzone.gif" width="700px"><br><br>
- paste the point as value of variable `roi_point`

In [4]:
utility = utils.Utils()

# paste the polygon point here!
roi_point = np.array([[32, 51], [244, 46], [388, 404], [51, 408]]) # CHANGE TO YOUR OWN POLYGON

origin_frame = cv2.imread("base_roi.jpg")

# set polygon ROI into utility instance
h, w = origin_frame.shape[:2]
utility.set_roi(roi_point, (h,w))

In [ ]:
# load trained Yolo V8 ONNX
model = "model/yolov8s.onnx"
net = cv2.dnn.readNetFromONNX(model)

# load video
cap = cv2.VideoCapture("mall.mp4")
           
# iterate for each frame in video
while cap.isOpened():
    
    # get image on each frame
    ret, frame = cap.read()
    if not ret:
        break
    #frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)

    # convert to blob
    resize_h, resize_w = 320, 320 # use smaller image size to reduce computation 
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (resize_w, resize_h), (0, 0, 0), swapRB=True, crop=False)

    # do a net forward (inferencing)
    net.setInput(blob)
    output = net.forward()

    # do a postprocessing
    frame = utility.postprocess_onnx(output, frame, classNames, confThreshold = 0.6, nmsThreshold = 0.4, font_size=0.5, 
                        color=(255,127,0), text_color=(255,255,255), input_size=[resize_h, resize_w])

    # do print count of object
    utility.print_object_count()

    # do draw count of object in image
    h, w, __ = frame.shape
    x0, y0 = int(0.75*w), int(0.1*h) # define counter box in 75% width , 10% height
    frame = utility.draw_object_count(frame, x0, y0, font_size=0.4, color=(0,127,255), text_color=(255,255,255), padding = 10)

    # show result
    cv2.imshow('Frame',frame)

    # wait 25ms per frame and close using 'q' 
    if cv2.waitKey(1) == ord('q'):
          break


cap.release()
cv2.destroyAllWindows()